# German Power Market Monitor Dashboard
### Authors:
**Hemanth Jadiswami Prabhakaran [Mat# 7026000]**

**Manoj Kumar Prabhakaran [Mat# 7026006]**
## Project Overview
This interactive dashboard analyzes and visualizes German energy market data from [SMARD](https://www.smard.de/en), Germany's official power market data platform. The dashboard presents insights into actual power generation across different sources and day-ahead electricity prices for Germany and neighboring countries.

## Data Sources
Data can be downloaded from the [SMARD Download Center](https://www.smard.de/en/downloadcenter/download-market-data/)

### Files Used in This Project:
1. **Generation Data**: `Actual_generation_202411010000_202412010000_Day.csv`
   - Daily power generation by source
   - Time period: November 2024
   - Units: MWh

2. **Price Data**: `Day-ahead_prices_202411010000_202412010000_Day.csv`
   - Daily electricity prices
   - Time period: November 2024
   - Units: €/MWh
   - Coverage: Germany and neighboring markets

## Dashboard Features
- Interactive visualization of energy generation mix
- Day-ahead price comparison across markets
- Date range selection
- Source/country filtering
- Statistical summaries

## Required Packages
```python
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
```

## Dashboard Access
Once you run this notebook, the dashboard will be available at:
- Local access: http://127.0.0.1:8050
- Network access: http://[your-ip-address]:8050

---
*Note: The code below implements an interactive dashboard using Dash. Execute all cells to launch the dashboard.*

In [11]:
# German Power Market Monitor Dashboard

# Author: 
# Hemanth Jadiswami Prabhakaran [Mat# 7026000] 
# Manoj Kumar Prabhakaran [Mat# 7026006]

# Version: 1.0
# Created: 29 Jan 2025
# Description: A comprehensive dashboard for monitoring German energy market data, 
# including actual generation by source and day-ahead electricity prices across German/European markets.

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize Dash application with callback exception handling enabled
# This allows for callbacks to be triggered even if some components are not yet loaded
app = dash.Dash(__name__, suppress_callback_exceptions=True)

def load_generation_data():
    """
    Loads and preprocesses energy generation data from CSV.
    
    Returns:
        pandas.DataFrame: Processed generation data with:
            - Datetime index
            - Cleaned column names (removed units and annotations)
            - Numeric values converted to float type
            - Missing values ('-') replaced with '0'
            - Comma decimal separators handled
    """
    df = pd.read_csv('Actual_generation_202411010000_202412010000_Day.csv', 
                     sep=';')
    
    # Parse dates and set as index for time series operations
    df['Start date'] = pd.to_datetime(df['Start date'])
    
    # Clean column headers by removing measurement units and annotations
    df.columns = [col.split(' [')[0] if '[' in col else col for col in df.columns]
    
    # Process numeric columns: handle missing values and convert to float
    numeric_columns = df.columns.difference(['Start date', 'End date'])
    for col in numeric_columns:
        df[col] = df[col].replace('-', '0')  
        df[col] = df[col].str.replace(',', '').astype(float)
    
    return df

def load_price_data():
    """
    Loads and preprocesses day-ahead electricity price data from CSV.
    
    Returns:
        pandas.DataFrame: Processed price data with:
            - Datetime index
            - Cleaned column names (removed units and annotations)
            - Numeric values converted to float type
            - Missing values ('-') replaced with '0'
            - Comma decimal separators handled
    """
    df = pd.read_csv('Day-ahead_prices_202411010000_202412010000_Day.csv', 
                     sep=';')
    
    df['Start date'] = pd.to_datetime(df['Start date'])
    
    # Remove units and annotations from column names
    df.columns = [col.split(' [')[0] if '[' in col else col for col in df.columns]
    
    # Handle numeric columns with special attention to decimal separator
    numeric_columns = df.columns.difference(['Start date', 'End date'])
    for col in numeric_columns:
        df[col] = df[col].replace('-', '0')  
        if isinstance(df[col].iloc[0], str):
            df[col] = df[col].str.replace(',', '.').astype(float)
    
    return df

# Color scheme definition for energy sources visualization
# Each source has a unique, visually distinguishable color
colors = {
    'Biomass': '#8884d8',
    'Hydropower': '#82ca9d',
    'Wind offshore': '#8dd1e1',
    'Wind onshore': '#82b1ff',
    'Photovoltaics': '#ffc658',
    'Other renewable': '#a4de6c',
    'Nuclear': '#d88484',
    'Lignite': '#97746a',
    'Hard coal': '#666666',
    'Fossil gas': '#ff8a65',
    'Hydro pumped storage': '#ba68c8',
    'Other conventional': '#78909c'
}

# Load initial data for setting up date ranges and options
initial_gen_df = load_generation_data()
initial_price_df = load_price_data()

# App layout
app.layout = html.Div([
    html.H1('German Energy Generation & Price Dashboard - November 2024',
            style={'textAlign': 'center', 'margin-bottom': '20px'}),
    
    dcc.Tabs(id='tabs', value='tab-1', children=[
        dcc.Tab(label='Actual Energy Generation', value='tab-1'),
        dcc.Tab(label='Day-ahead Energy Prices', value='tab-2'),
    ]),
    
    html.Div(id='tabs-content')
])

# Generation tab layout
def generate_generation_tab():
    return html.Div([
        html.Div([
            html.H3('Select Date Range'),
            dcc.DatePickerRange(
                id='generation-date-picker',
                min_date_allowed=initial_gen_df['Start date'].min(),
                max_date_allowed=initial_gen_df['Start date'].max(),
                start_date=initial_gen_df['Start date'].min(),
                end_date=initial_gen_df['Start date'].max(),
                display_format='YYYY-MM-DD'
            ),
        ], style={'width': '100%', 'margin-bottom': '20px'}),
        
        html.Div([
            html.H3('Select Energy Sources'),
            dcc.Checklist(
                id='energy-sources',
                options=[{'label': source, 'value': source} 
                        for source in colors.keys()],
                value=list(colors.keys()),
                inline=True,
                style={'margin-bottom': '20px'}
            )
        ], style={'width': '100%', 'margin-bottom': '20px'}),
        
        dcc.Graph(id='generation-chart'),
        
        html.Div([
            html.H3('Generation Summary', 
                   style={'margin-top': '20px', 'margin-bottom': '10px'}),
            html.Div(id='generation-stats')
        ])
    ], style={'width': '90%', 'margin': 'auto'})

# Price tab layout
def generate_price_tab():
    return html.Div([
        html.Div([
            html.H3('Select November Date Range'),
            dcc.DatePickerRange(
                id='price-date-picker',
                min_date_allowed=initial_price_df['Start date'].min(),
                max_date_allowed=initial_price_df['Start date'].max(),
                start_date=initial_price_df['Start date'].min(),
                end_date=initial_price_df['Start date'].max(),
                display_format='YYYY-MM-DD'
            ),
        ], style={'width': '100%', 'margin-bottom': '20px'}),
        
        html.Div([
            html.H3('Select Countries'),
            dcc.Dropdown(
                id='country-selector',
                options=[
                    {'label': col, 'value': col} 
                    for col in initial_price_df.columns 
                    if col not in ['Start date', 'End date']
                ],
                value=['Germany/Luxembourg'],
                multi=True
            )
        ], style={'width': '100%', 'margin-bottom': '20px'}),
        
        dcc.Graph(id='price-chart'),
        
        html.Div([
            html.H3('Price Statistics', 
                   style={'margin-top': '20px', 'margin-bottom': '10px'}),
            html.Div(id='price-stats')
        ])
    ], style={'width': '90%', 'margin': 'auto'})

# Callback for tabs
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return generate_generation_tab()
    elif tab == 'tab-2':
        return generate_price_tab()

# Callback for generation chart
@app.callback(
    [Output('generation-chart', 'figure'),
     Output('generation-stats', 'children')],
    [Input('energy-sources', 'value'),
     Input('generation-date-picker', 'start_date'),
     Input('generation-date-picker', 'end_date')]
)
def update_generation_graph(selected_sources, start_date, end_date):
    df = load_generation_data()
    
    # Filter data based on selected date range
    mask = (df['Start date'] >= start_date) & (df['Start date'] <= end_date)
    df_filtered = df.loc[mask]
    
    # Create stacked area chart
    fig = go.Figure()
    
    for source in selected_sources:
        fig.add_trace(go.Scatter(
            x=df_filtered['Start date'],
            y=df_filtered[source],
            name=source,
            mode='lines',
            stackgroup='one',
            line=dict(width=0.5),
            fillcolor=colors[source]
        ))
    
    fig.update_layout(
        title=dict(
            text='Energy Generation Over Time',
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top',
            pad=dict(b=20)
        ),
        xaxis_title='Date',
        yaxis_title='Generation (MWh)',
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(t=100)
    )
    
    # Create summary statistics
    summary_stats = []
    for source in selected_sources:
        total_generation = float(df_filtered[source].sum())
        summary_stats.append(
            html.Div([
                html.H4(source),
                html.P('{:,.0f} MWh'.format(total_generation))
            ], style={
                'width': '30%', 
                'display': 'inline-block',
                'margin': '10px',
                'padding': '10px',
                'backgroundColor': '#f8f9fa',
                'borderRadius': '5px',
                'textAlign': 'center'
            })
        )
    
    return fig, html.Div(summary_stats, style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center'})

# Callback for price chart
@app.callback(
    [Output('price-chart', 'figure'),
     Output('price-stats', 'children')],
    [Input('country-selector', 'value'),
     Input('price-date-picker', 'start_date'),
     Input('price-date-picker', 'end_date')]
)
def update_price_graph(selected_countries, start_date, end_date):
    df = load_price_data()
    
    # Filter data based on selected date range
    mask = (df['Start date'] >= start_date) & (df['Start date'] <= end_date)
    df_filtered = df.loc[mask]
    
    # Create line chart for prices
    fig = go.Figure()
    
    for country in selected_countries:
        fig.add_trace(go.Scatter(
            x=df_filtered['Start date'],
            y=df_filtered[country],
            name=country,
            mode='lines',
            line=dict(width=2, shape='hv'),  # 'hv' creates horizontal-vertical steps
            line_shape='hv'  # Reinforcing the step shape
        ))
    
    fig.update_layout(
        title=dict(
            text='Energy Prices Over Time',
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top',
            pad=dict(b=20)
        ),
        xaxis_title='Date',
        yaxis_title='Price (€/MWh)',
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(t=100),
        xaxis=dict(
            gridcolor='lightgrey',
            showgrid=True,
            tickformat='%d. %b'   
        ),
        yaxis=dict(
            gridcolor='lightgrey',
            showgrid=True,
            range=[0, 250]  
        ),
        plot_bgcolor='white',   
        paper_bgcolor='white'   
    )
    
    # Create price statistics
    stats = []
    for country in selected_countries:
        avg_price = float(df_filtered[country].mean())
        max_price = float(df_filtered[country].max())
        min_price = float(df_filtered[country].min())
        
        stats.append(
            html.Div([
                html.H4(country),
                html.P([
                    f'Average: {avg_price:.2f} €/MWh',
                    html.Br(),
                    f'Max: {max_price:.2f} €/MWh',
                    html.Br(),
                    f'Min: {min_price:.2f} €/MWh'
                ])
            ], style={
                'width': '30%', 
                'display': 'inline-block',
                'margin': '10px',
                'padding': '10px',
                'backgroundColor': '#f8f9fa',
                'borderRadius': '5px',
                'textAlign': 'center'
            })
        )
    
    return fig, html.Div(stats, style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center'})

if __name__ == '__main__':
    app.run_server(debug=True)